# Module 01: Metadata Management

**Difficulty**: ⭐⭐
**Estimated Time**: 45 minutes
**Prerequisites**: [Module 00: Music Library Manager](00_music_library_manager.ipynb)

## Learning Objectives
By the end of this notebook, you will be able to:
1. Extract and display ID3 metadata tags from audio files
2. Search your library by artist, album, title, or genre (not just filename)
3. Identify files with missing or incomplete metadata
4. Parse filenames to extract metadata information
5. Batch update metadata tags across multiple files
6. Analyze metadata quality and completeness

## Overview
This module extends the basic music library manager with powerful metadata capabilities. Instead of relying solely on filenames, you'll be able to work with embedded ID3 tags that contain rich information about your music:

- **Artist, Album, Title**: Core identification tags
- **Year, Genre**: Categorization information
- **Track Number**: Organization within albums
- **And more**: Comments, album artist, composer, etc.

### Why Metadata Matters
- **Better Organization**: Organize by actual artist/album, not filename guesses
- **Accurate Search**: Find "The Beatles" even if filename says "Beatles" or "beatles"
- **Quality Control**: Identify files with missing or incorrect information
- **Consistency**: Standardize tags across your entire library

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import os
from pathlib import Path
from collections import defaultdict, Counter
from typing import List, Dict, Optional, Tuple
import re
from datetime import datetime

# Metadata handling
import mutagen
from mutagen.easyid3 import EasyID3
from mutagen.id3 import ID3
from mutagen.mp3 import MP3
from mutagen.flac import FLAC
from mutagen.mp4 import MP4
from mutagen import File

# For better display
import pandas as pd
from IPython.display import display, HTML
import warnings

# Configuration
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore', category=UserWarning)

print("✓ Libraries imported successfully")

In [ ]:
# Configure your music library path
MUSIC_LIBRARY_PATH = Path('../../music')

# Supported audio file extensions
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.wav', '.m4a', '.aac', '.ogg', '.wma', '.opus'}

# Critical metadata tags (for completeness checking)
CRITICAL_TAGS = ['artist', 'album', 'title']
IMPORTANT_TAGS = ['date', 'genre', 'tracknumber']

# Verify the path exists
if not MUSIC_LIBRARY_PATH.exists():
    print(f"⚠️ Warning: Music library path does not exist: {MUSIC_LIBRARY_PATH.absolute()}")
    MUSIC_LIBRARY_PATH.mkdir(parents=True, exist_ok=True)
else:
    print(f"✓ Music library path: {MUSIC_LIBRARY_PATH.absolute()}")

## 2. Metadata Extraction Functions

In [ ]:
def extract_metadata(file_path: Path) -> Dict[str, Optional[str]]:
    """
    Extract metadata from an audio file using mutagen.
    
    Args:
        file_path: Path to the audio file
    
    Returns:
        Dictionary containing metadata tags
    """
    metadata = {
        'artist': None,
        'album': None,
        'title': None,
        'date': None,
        'genre': None,
        'tracknumber': None,
        'albumartist': None,
        'composer': None,
        'comment': None
    }
    
    try:
        # Use mutagen to read the file
        audio = File(file_path, easy=True)
        
        if audio is None:
            return metadata
        
        # Extract common tags
        for tag in metadata.keys():
            if tag in audio:
                # Mutagen returns lists, get first value
                value = audio[tag]
                if isinstance(value, list) and len(value) > 0:
                    metadata[tag] = str(value[0])
                else:
                    metadata[tag] = str(value)
    
    except Exception as e:
        # If metadata extraction fails, return empty metadata
        pass
    
    return metadata


def get_metadata_summary(file_path: Path) -> str:
    """
    Get a human-readable summary of file metadata.
    
    Returns:
        Formatted string with metadata information
    """
    metadata = extract_metadata(file_path)
    
    lines = []
    lines.append(f"File: {file_path.name}")
    lines.append("-" * 50)
    
    for tag, value in metadata.items():
        if value:
            lines.append(f"{tag.capitalize()}: {value}")
        else:
            lines.append(f"{tag.capitalize()}: [Missing]")
    
    return "\n".join(lines)


def scan_library_with_metadata(library_path: Path) -> List[Dict]:
    """
    Scan the music library and include metadata for each file.
    Enhanced version of the basic scanner.
    
    Returns:
        List of dictionaries containing file information and metadata
    """
    music_files = []
    
    print(f"Scanning library: {library_path}")
    
    # Walk through all directories and subdirectories
    file_count = 0
    for root, dirs, files in os.walk(library_path):
        for file in files:
            file_path = Path(root) / file
            
            # Check if it's an audio file
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                file_count += 1
                
                # Progress indicator for large libraries
                if file_count % 100 == 0:
                    print(f"  Processed {file_count} files...")
                
                # Get file information
                stat_info = file_path.stat()
                
                # Extract metadata
                metadata = extract_metadata(file_path)
                
                # Combine file info and metadata
                file_info = {
                    'filename': file,
                    'path': str(file_path),
                    'folder': str(file_path.parent.relative_to(library_path)),
                    'size_mb': stat_info.st_size / (1024 * 1024),
                    'modified': datetime.fromtimestamp(stat_info.st_mtime),
                    'extension': file_path.suffix.lower(),
                    # Metadata fields
                    'artist': metadata.get('artist'),
                    'album': metadata.get('album'),
                    'title': metadata.get('title'),
                    'date': metadata.get('date'),
                    'genre': metadata.get('genre'),
                    'tracknumber': metadata.get('tracknumber'),
                    'albumartist': metadata.get('albumartist'),
                    'composer': metadata.get('composer')
                }
                
                music_files.append(file_info)
    
    print(f"✓ Scan complete: {file_count} audio files found")
    return music_files

print("✓ Metadata extraction functions loaded")

## 3. Enhanced Search Functions

In [ ]:
def search_by_metadata(music_files: List[Dict], 
                       query: str,
                       fields: List[str] = None,
                       case_sensitive: bool = False) -> pd.DataFrame:
    """
    Search for songs by metadata fields (artist, album, title, etc.).
    
    Args:
        music_files: List of music file dictionaries
        query: Search term
        fields: List of fields to search in. If None, searches all metadata fields.
        case_sensitive: Whether search should be case-sensitive
    
    Returns:
        DataFrame with matching songs
    """
    # Default: search in all metadata fields and filename
    if fields is None:
        fields = ['filename', 'artist', 'album', 'title', 'genre', 'albumartist']
    
    if not case_sensitive:
        query = query.lower()
    
    results = []
    for file in music_files:
        # Check each specified field
        for field in fields:
            value = file.get(field)
            if value is not None:
                # Convert to string and apply case sensitivity
                value_str = str(value) if case_sensitive else str(value).lower()
                
                if query in value_str:
                    results.append(file)
                    break  # Found a match, no need to check other fields
    
    if results:
        df = pd.DataFrame(results)
        df['size_mb'] = df['size_mb'].round(2)
        return df
    else:
        return pd.DataFrame()


def search_by_artist(music_files: List[Dict], artist: str) -> pd.DataFrame:
    """
    Search for all songs by a specific artist.
    """
    return search_by_metadata(music_files, artist, fields=['artist', 'albumartist'])


def search_by_album(music_files: List[Dict], album: str) -> pd.DataFrame:
    """
    Search for all songs from a specific album.
    """
    return search_by_metadata(music_files, album, fields=['album'])


def search_by_genre(music_files: List[Dict], genre: str) -> pd.DataFrame:
    """
    Search for all songs of a specific genre.
    """
    return search_by_metadata(music_files, genre, fields=['genre'])


def search_by_year(music_files: List[Dict], year: str) -> pd.DataFrame:
    """
    Search for all songs from a specific year.
    """
    return search_by_metadata(music_files, str(year), fields=['date'])

print("✓ Enhanced search functions loaded")

## 4. Metadata Quality Analysis

In [ ]:
def calculate_metadata_completeness(file_info: Dict) -> Dict:
    """
    Calculate metadata completeness score for a file.
    
    Returns:
        Dictionary with completeness metrics
    """
    # Check critical tags (artist, album, title)
    critical_present = sum(1 for tag in CRITICAL_TAGS if file_info.get(tag))
    critical_score = (critical_present / len(CRITICAL_TAGS)) * 100
    
    # Check important tags (date, genre, tracknumber)
    important_present = sum(1 for tag in IMPORTANT_TAGS if file_info.get(tag))
    important_score = (important_present / len(IMPORTANT_TAGS)) * 100
    
    # Overall score (weighted: critical 70%, important 30%)
    overall_score = (critical_score * 0.7) + (important_score * 0.3)
    
    return {
        'critical_tags': f"{critical_present}/{len(CRITICAL_TAGS)}",
        'important_tags': f"{important_present}/{len(IMPORTANT_TAGS)}",
        'overall_score': round(overall_score, 1),
        'missing_critical': [tag for tag in CRITICAL_TAGS if not file_info.get(tag)],
        'missing_important': [tag for tag in IMPORTANT_TAGS if not file_info.get(tag)]
    }


def find_files_with_missing_metadata(music_files: List[Dict], 
                                      tags: List[str] = None) -> pd.DataFrame:
    """
    Find files with missing metadata tags.
    
    Args:
        tags: List of tags to check. If None, checks critical tags.
    
    Returns:
        DataFrame with files that have missing tags
    """
    if tags is None:
        tags = CRITICAL_TAGS
    
    files_with_issues = []
    
    for file in music_files:
        missing_tags = [tag for tag in tags if not file.get(tag)]
        
        if missing_tags:
            file_copy = file.copy()
            file_copy['missing_tags'] = ', '.join(missing_tags)
            files_with_issues.append(file_copy)
    
    if files_with_issues:
        df = pd.DataFrame(files_with_issues)
        return df
    else:
        return pd.DataFrame()


def get_metadata_quality_report(music_files: List[Dict]) -> Dict:
    """
    Generate a comprehensive metadata quality report.
    
    Returns:
        Dictionary with quality statistics
    """
    if not music_files:
        return {'error': 'No files to analyze'}
    
    total_files = len(music_files)
    
    # Count files with each tag
    tag_coverage = {}
    for tag in CRITICAL_TAGS + IMPORTANT_TAGS:
        count = sum(1 for f in music_files if f.get(tag))
        percentage = (count / total_files) * 100
        tag_coverage[tag] = {
            'count': count,
            'percentage': round(percentage, 1)
        }
    
    # Calculate overall completeness
    scores = [calculate_metadata_completeness(f)['overall_score'] for f in music_files]
    avg_score = sum(scores) / len(scores)
    
    # Categorize by quality
    excellent = sum(1 for s in scores if s >= 90)
    good = sum(1 for s in scores if 70 <= s < 90)
    fair = sum(1 for s in scores if 50 <= s < 70)
    poor = sum(1 for s in scores if s < 50)
    
    return {
        'total_files': total_files,
        'average_completeness': round(avg_score, 1),
        'tag_coverage': tag_coverage,
        'quality_distribution': {
            'excellent (90-100%)': excellent,
            'good (70-89%)': good,
            'fair (50-69%)': fair,
            'poor (<50%)': poor
        }
    }

print("✓ Metadata quality analysis functions loaded")

## 5. Filename Parsing

In [ ]:
def parse_filename_to_metadata(filename: str) -> Dict[str, Optional[str]]:
    """
    Attempt to parse metadata from filename patterns.
    Supports common patterns like:
    - "Artist - Title.mp3"
    - "Artist - Album - Title.mp3"
    - "01 - Artist - Title.mp3"
    - "Track Number - Title.mp3"
    
    Returns:
        Dictionary with parsed metadata
    """
    metadata = {
        'artist': None,
        'album': None,
        'title': None,
        'tracknumber': None
    }
    
    # Remove file extension
    name_without_ext = Path(filename).stem
    
    # Common patterns to try
    patterns = [
        # "Artist - Album - Title"
        r'^(?P<artist>[^-]+)\s*-\s*(?P<album>[^-]+)\s*-\s*(?P<title>.+)$',
        # "Artist - Title"
        r'^(?P<artist>[^-]+)\s*-\s*(?P<title>.+)$',
        # "01 - Artist - Title" or "01. Artist - Title"
        r'^(?P<tracknumber>\d+)[\s\.-]+(?P<artist>[^-]+)\s*-\s*(?P<title>.+)$',
        # "01 - Title" or "01. Title"
        r'^(?P<tracknumber>\d+)[\s\.-]+(?P<title>.+)$',
    ]
    
    for pattern in patterns:
        match = re.match(pattern, name_without_ext.strip())
        if match:
            for key, value in match.groupdict().items():
                if value:
                    metadata[key] = value.strip()
            break
    
    # If no pattern matched, use entire filename as title
    if not any(metadata.values()):
        metadata['title'] = name_without_ext
    
    return metadata


def suggest_metadata_from_filename(file_info: Dict) -> Dict:
    """
    Suggest metadata values based on filename parsing.
    Only suggests for fields that are currently empty.
    
    Returns:
        Dictionary with suggested metadata updates
    """
    parsed = parse_filename_to_metadata(file_info['filename'])
    suggestions = {}
    
    for key, value in parsed.items():
        # Only suggest if current metadata is missing and we parsed something
        if value and not file_info.get(key):
            suggestions[key] = value
    
    return suggestions

print("✓ Filename parsing functions loaded")

## 6. Metadata Writing and Batch Updates

In [ ]:
def update_metadata(file_path: Path, metadata: Dict[str, str], 
                    dry_run: bool = True) -> Dict:
    """
    Update metadata tags for a single file.
    
    Args:
        file_path: Path to the audio file
        metadata: Dictionary of tag:value pairs to update
        dry_run: If True, only show what would be updated
    
    Returns:
        Dictionary with update status
    """
    result = {
        'file': str(file_path),
        'status': 'Would update' if dry_run else 'Updated',
        'updates': metadata
    }
    
    if not dry_run:
        try:
            # Use mutagen to update tags
            audio = File(file_path, easy=True)
            
            if audio is None:
                result['status'] = 'Failed - unsupported format'
                return result
            
            # Update each tag
            for tag, value in metadata.items():
                if value:  # Only update if value is not None/empty
                    audio[tag] = str(value)
            
            # Save changes
            audio.save()
            result['status'] = 'Updated'
            
        except Exception as e:
            result['status'] = f'Failed - {str(e)}'
    
    return result


def batch_update_from_filename(music_files: List[Dict], 
                                dry_run: bool = True) -> List[Dict]:
    """
    Batch update metadata by parsing filenames.
    Only updates files with missing metadata.
    
    Args:
        music_files: List of file information dictionaries
        dry_run: If True, only preview changes
    
    Returns:
        List of update actions
    """
    actions = []
    
    for file in music_files:
        # Get suggestions from filename
        suggestions = suggest_metadata_from_filename(file)
        
        if suggestions:
            result = update_metadata(
                Path(file['path']), 
                suggestions,
                dry_run=dry_run
            )
            result['filename'] = file['filename']
            actions.append(result)
    
    return actions


def batch_update_metadata(music_files: List[Dict], 
                          tag: str, 
                          value: str,
                          dry_run: bool = True) -> List[Dict]:
    """
    Batch update a specific tag across multiple files.
    
    Args:
        music_files: List of files to update
        tag: Tag name to update (e.g., 'artist', 'album')
        value: New value for the tag
        dry_run: If True, only preview changes
    
    Returns:
        List of update actions
    """
    actions = []
    
    for file in music_files:
        result = update_metadata(
            Path(file['path']),
            {tag: value},
            dry_run=dry_run
        )
        result['filename'] = file['filename']
        actions.append(result)
    
    return actions

print("✓ Metadata writing functions loaded")

---
## 7. Usage Examples

### 7.1 Scan Library with Metadata

In [ ]:
# Scan the library and extract metadata
print("Scanning library with metadata extraction...\n")
all_music_files = scan_library_with_metadata(MUSIC_LIBRARY_PATH)

print(f"\nFound {len(all_music_files)} audio files")

# Display sample with metadata
if all_music_files:
    sample_df = pd.DataFrame(all_music_files[:10])
    display_cols = ['filename', 'artist', 'album', 'title', 'genre', 'date']
    display(sample_df[display_cols])
else:
    print("\nNo music files found. Add some music to test!")

### 7.2 View Detailed Metadata for a File

In [ ]:
# Show detailed metadata for first file (if exists)
if all_music_files:
    sample_file = Path(all_music_files[0]['path'])
    print(get_metadata_summary(sample_file))
else:
    print("No files to display metadata for.")

### 7.3 Search by Metadata

In [ ]:
# Example: Search by artist
artist_query = "beatles"  # Change this to search for a specific artist

print(f"Searching for artist: '{artist_query}'...\n")
results = search_by_artist(all_music_files, artist_query)

if len(results) > 0:
    print(f"Found {len(results)} songs:\n")
    display(results[['filename', 'artist', 'album', 'title', 'date']])
else:
    print("No songs found for this artist.")
    print("\n💡 Tip: Search works on metadata, not just filenames!")

In [ ]:
# Search by album
album_query = "your_album_name"  # Change this

results = search_by_album(all_music_files, album_query)
if len(results) > 0:
    print(f"\nFound {len(results)} songs from album '{album_query}':")
    display(results[['filename', 'artist', 'title', 'tracknumber']])
else:
    print(f"No songs found from album '{album_query}'")

In [ ]:
# Search by genre
genre_query = "rock"  # Change this

results = search_by_genre(all_music_files, genre_query)
if len(results) > 0:
    print(f"\nFound {len(results)} {genre_query} songs:")
    display(results[['filename', 'artist', 'album', 'genre']].head(20))
else:
    print(f"No {genre_query} songs found")

### 7.4 Metadata Quality Report

In [ ]:
# Generate metadata quality report
if all_music_files:
    print("📊 Metadata Quality Report")
    print("=" * 60 + "\n")
    
    report = get_metadata_quality_report(all_music_files)
    
    print(f"Total Files: {report['total_files']}")
    print(f"Average Completeness: {report['average_completeness']}%\n")
    
    print("Tag Coverage:")
    for tag, stats in report['tag_coverage'].items():
        print(f"  {tag.capitalize()}: {stats['count']} files ({stats['percentage']}%)")
    
    print("\nQuality Distribution:")
    for category, count in report['quality_distribution'].items():
        print(f"  {category}: {count} files")
else:
    print("No files to analyze.")

### 7.5 Find Files with Missing Metadata

In [ ]:
# Find files missing critical metadata
if all_music_files:
    print("Finding files with missing critical metadata (artist, album, title)...\n")
    
    missing = find_files_with_missing_metadata(all_music_files)
    
    if len(missing) > 0:
        print(f"⚠️ Found {len(missing)} files with missing metadata:\n")
        display(missing[['filename', 'artist', 'album', 'title', 'missing_tags']].head(20))
        
        print(f"\n💡 Showing first 20 of {len(missing)} files")
        print("💡 You can auto-tag these using batch_update_from_filename()")
    else:
        print("✓ All files have complete critical metadata!")
else:
    print("No files to check.")

### 7.6 Parse Metadata from Filenames

In [ ]:
# Example: Parse a filename to extract metadata
example_filenames = [
    "The Beatles - Abbey Road - Come Together.mp3",
    "Pink Floyd - Wish You Were Here.mp3",
    "01 - Led Zeppelin - Stairway to Heaven.mp3",
    "05. Bohemian Rhapsody.mp3"
]

print("Filename Parsing Examples:\n" + "=" * 60 + "\n")

for filename in example_filenames:
    parsed = parse_filename_to_metadata(filename)
    print(f"Filename: {filename}")
    print(f"Parsed metadata:")
    for key, value in parsed.items():
        if value:
            print(f"  {key}: {value}")
    print()

### 7.7 Batch Update from Filenames (Dry Run)

In [ ]:
# Preview: Auto-tag files with missing metadata using filename parsing
if all_music_files:
    print("Preview: Auto-tagging from filenames (dry run)...\n")
    
    actions = batch_update_from_filename(all_music_files, dry_run=True)
    
    if actions:
        print(f"Would update {len(actions)} files:\n")
        df = pd.DataFrame(actions[:20])  # Show first 20
        display(df[['filename', 'updates', 'status']])
        
        print(f"\n💡 Showing first 20 of {len(actions)} files")
        print("\n💡 To actually update, run:")
        print("   actions = batch_update_from_filename(all_music_files, dry_run=False)")
    else:
        print("✓ No files need auto-tagging (all have complete metadata or filenames can't be parsed)")
else:
    print("No files to update.")

### 7.8 Manual Metadata Update Example

In [ ]:
# Example: Update album name for all songs from a specific artist
# Uncomment to use:

# artist_to_update = "Artist Name"
# new_album_name = "Album Name"
#
# # Find all songs by this artist
# artist_songs = search_by_artist(all_music_files, artist_to_update)
#
# if len(artist_songs) > 0:
#     print(f"Found {len(artist_songs)} songs by {artist_to_update}")
#     
#     # Preview update
#     actions = batch_update_metadata(
#         artist_songs.to_dict('records'),
#         tag='album',
#         value=new_album_name,
#         dry_run=True
#     )
#     
#     display(pd.DataFrame(actions))
#     
#     # To actually update, change dry_run=False

print("💡 Example code above shows how to batch update a specific tag")
print("💡 Uncomment and customize to use")

## 8. Summary

### What We've Learned

In this module, we've added powerful metadata management capabilities:

1. **Metadata Extraction**: Read ID3 tags from audio files (artist, album, title, etc.)
2. **Enhanced Search**: Search by metadata fields, not just filenames
3. **Quality Analysis**: Identify files with missing or incomplete metadata
4. **Filename Parsing**: Extract metadata from common filename patterns
5. **Batch Updates**: Update tags for multiple files at once
6. **Metadata Writing**: Modify and save metadata tags

### Key Functions

**Extraction:**
- `extract_metadata()` - Get all tags from a file
- `scan_library_with_metadata()` - Scan library with metadata

**Search:**
- `search_by_metadata()` - Search across metadata fields
- `search_by_artist()` / `search_by_album()` / `search_by_genre()` - Specialized searches

**Quality:**
- `calculate_metadata_completeness()` - Score metadata quality
- `find_files_with_missing_metadata()` - Find incomplete files
- `get_metadata_quality_report()` - Full quality report

**Updates:**
- `update_metadata()` - Update tags for a single file
- `batch_update_from_filename()` - Auto-tag from filenames
- `batch_update_metadata()` - Update specific tag across files

### Best Practices

1. **Always use dry-run first**: Preview changes before applying
2. **Backup before batch updates**: Metadata changes are permanent
3. **Validate after updates**: Re-scan to verify changes
4. **Standardize naming**: Use consistent artist/album names
5. **Regular quality checks**: Run quality reports periodically

### Next Steps

Continue to:
- **Module 02: Audio Analysis** - Analyze bitrate, quality, duration
- **Module 04: Advanced Organization** - Organize by Artist/Album structure
- **Module 05: API Integration** - Auto-tag from MusicBrainz

### Additional Resources

- [Mutagen Documentation](https://mutagen.readthedocs.io/)
- [ID3 Tag Specifications](https://id3.org/)
- [Audio Metadata Best Practices](https://en.wikipedia.org/wiki/ID3)